Week 2 Assignment: Association Rule Mining Assignment
==============

In this assignment we will be be identifying frequent itemsets from an online retail dataset using the FP-Growth Algorithm in Pyspark.

In [513]:
import pyspark 
from pyspark.mllib.fpm import FPGrowth

In [514]:
sc = pyspark.SparkContext()
data = sc.textFile("data/OnlineRetail.csv")

In [515]:
#Defines the function groupProductsOrders that groups StockCodes(Product) of the same InvoiceNo(Orders) in a python list
#Each row represents an order and products are grouped together in a string separated by spaces

def groupProductsOrders(header, dataset, rows_to_analyse, order_ix, product_ix, nr_orders, separator):
    a = 0
    last_order = ''
    index = -1
    index_limit = nr_orders
    result = []
    space = ''
    for i in dataset.take(rows_to_analyse):
        if (a == 0 and header == True):
            a = a + 1
            continue
        else:    
            list_aux = i.split(separator)

            if last_order != list_aux[order_ix]:            
                last_order = list_aux[order_ix]
                index = index+1
                if index == index_limit:
                    break
                space = ''
                result.append('')

            result[index] += space + list_aux[product_ix]
            space = ' '
            a = a +1    
    
    return result

In [516]:
#Grouping StockCodes of the same InvoiceNo
#Defining parameters

invoiceNo_index = 0   #invoiceNo = column 0
stockCode_index = 1   #stockCode = column 1
transactions = 5      #Return first 5 transactions/orders of the dataset
skipHeader = True     #Ignore header
rowsData = 1000       #Consider 1000 rows of the dataset
separatorData = ';'   #Separator = ;

top5Invoices = groupProductsOrders(skipHeader, data, rowsData, invoiceNo_index, stockCode_index, transactions, separatorData) 

In [517]:
transactions_temp = sc.parallelize(top5Invoices)
transactions = transactions_temp.map(lambda line: line.strip().split(' '))

In [518]:
model = FPGrowth.train(transactions, minSupport = 0.15, numPartitions = 10)

In [519]:
result_final = model.freqItemsets().collect()

**1) How many frequent itemsets were found with minimum support of .15?**

In [520]:
print(len(result_final))

4241


**Answer:** There are **4,241** frequent itemsets

**2) How many association rules were identified with a minimum confidence of .4?**

In [521]:
freqItemsetsDict = {}

for i in range(0, len(result_final)):
    freqItemsetsDict[str(result_final[i].items)] = result_final[i].freq
    
minConfidence = .4
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)

for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.2f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.2f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next    

['22310'] => ['21777'] (support: 0.20 confidence: 1.00)
['22310'] => ['22623'] (support: 0.20 confidence: 1.00)
['22310'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310'] => ['22748'] (support: 0.20 confidence: 1.00)
['22310'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310'] => ['84879'] (support: 0.20 confidence: 1.00)
['22310'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310'] => ['21754'] (support: 0.20 confidence: 1.00)
['22749'] => ['22623'] (support: 0.20 confidence: 1.00)
['22749'] => ['22310'] (support: 0.20 confidence: 1.00)
['22749'] => ['48187'] (support: 0.20 confidence: 1.00)
['22749'] => ['84969'] (support: 0.20 confidence: 1.00)
['22749'] => ['21754'] (support: 0.20 confidence: 1.00)
['22749'] => ['84879'] (support: 0.20 confidence: 1.00)
['22749'] => ['22748'] (support: 0.20 confidence

['48187', '22623', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['48187', '22623', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['48187', '22623', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['48187', '22623', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['48187', '22623', '22748'] => ['22310'] (support: 0.20 confidence: 1.00)
['48187', '22623', '22748'] => ['84879'] (support: 0.20 confidence: 1.00)
['48187', '22623', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['48187', '84969', '22623', '21777', '22745'] => ['22748'] (support: 0.20 confidence: 1.00)
['48187', '84969', '22623', '21777', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['48187', '84969', '22623', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['48187', '84969', '22623', '21777', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['48187', '84969', '22623', '21777', '22745'] => ['22310'] (support: 0.20 confidence: 1.00)
['48187', '84969', '22

['21754', '48187', '84969', '22623', '22622', '22749', '22748', '22745', '21755'] => ['84879'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22623', '22622', '22749', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22623', '22622', '22749', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22623', '22622', '22749', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22623', '22622', '22749', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22623', '22622', '22749', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22623', '22622', '22749', '21755'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22745'] => ['22748'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22745'] => ['22622'] (su

['48187', '84879', '84969', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['48187', '84879', '84969', '22749', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['48187', '84879', '84969', '22749', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22745', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22745', '21755'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22745', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22745', '21755'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22745', '21755'] => ['21754'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22622', '21777', '22749', '22745', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22622', '21777', '22749', 

['48187', '22310', '84879', '22623', '22622', '21777', '22749', '22748', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['48187', '22310', '84879', '22623', '22622', '21777', '22749', '22748', '22745'] => ['84969'] (support: 0.20 confidence: 1.00)
['48187', '22310', '84879', '22623', '22622', '21777', '22749', '22748', '22745'] => ['21754'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '22622', '21777', '22748', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '22622', '21777', '22748', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '22622', '21777', '22748', '22745'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '22622', '21777', '22748', '22745'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22622', '21777', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['22623', '22622', '21777', '22748'] => ['21755'] (suppor

...

['22310', '84879', '22622', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '22748'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '21777', '22748', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '21777', '22748', '22745'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '21777', '22748', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '21777', '22748', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '21777', '22748', '22745'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310', '84879', '22622', '21777', '22748', '22745'] => ['21754'] (support: 0.20 confidence: 1.00)
['48187', '84879', '22622', '21777', '21755'] => ['22310'] (su

['22623', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['48187'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84879', '84969', '22623', '22622', '21777', '22749', '21755'] => ['48187'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84879', '84969', '22623', '22622', '21777', '22749', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84879', '84969', '22623', '22622', '21777', '22749', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '48187', '84879', '22622', '22749', '21755'] => ['22310'] (support: 0.20 confidence: 1

['22310', '84879', '84969', '22622', '22749', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22622', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22622', '22749', '22748'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22622', '22749', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['21754', '84879', '84969', '22623', '22622', '22745'] => ['48187'] (support: 0.20 confidence: 1.00)
['21754', '84879', '84969', '22623', '22622', '22745'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84879', '84969', '22623', '22622', '22745'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '84879', '84969', '22623', '22622', '22745'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '84879', '84969', '22623', '22622', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '84879', '84969', '22623', '22622', '22745'] => ['21755'] (support: 0.20 confiden

['21754', '48187', '84969', '21777', '22748', '22745', '21755'] => ['84879'] (support: 0.20 confidence: 1.00)
['21754', '48187', '84879', '84969', '22623', '22622', '22749', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '48187', '84879', '84969', '22623', '22622', '22749', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '48187', '84879', '84969', '22623', '22622', '22749', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '48187', '84879', '84969', '22623', '22622', '22749', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22749', '22745', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22749', '22745', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22749', '22745', '21755'] => ['48187'] (support: 0.20 confidence: 1.00)
['22310', '84879', '84969', '22623', '21777', '22749', '2

['48187', '84969'] => ['22623'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['22748'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['22749'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['22745'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['22622'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['21755'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['22310'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['84879'] (support: 0.20 confidence: 1.00)
['48187', '84969'] => ['21754'] (support: 0.20 confidence: 1.00)
['48187', '22310', '84879', '84969', '22623', '22622', '22749', '22748'] => ['21777'] (support: 0.20 confidence: 1.00)
['48187', '22310', '84879', '84969', '22623', '22622', '22749', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['48187', '22310', '84879', '84969', '22623', '22622', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['48187', '22310', '84879', '84969', '22623', '22622', '22

In [522]:
print(len(rules))

25035


**Answer:** There are **25,035** rules with a minimum confidence of .4.

**3) What is the support and confidence of the following association rule: ['21754', '22748'] => ['22745']? Go beyond reporting the values, but include an interpretation of these evaluation metrics.**

**Answer:** Based on the results printed above we can see that the association rule ['21754', '22748'] => ['22745'] has support of .20 and confidence of 1.00.

That means that products '21754', '22748' and '22745' appear in 20% of the transactions.

The confidence of 1.00 indicates that 100% of the transactions that contain the products '21754'and '22748' also contain the product '22745'.